# CC-cost for Sep/22

This script runs a `sql` code to create a panel data, based on the `case_records_current` table, where we have each `ICD-10` member change of status, and the link between the status change with change in daly weights. 
The calculation involves a lot of cleaning,   such as desconsidering impetous changes of status or *cancelled* icds status as the first status input, etc, we leave that for the sql code, and here is just the script to 
run it and export to the redshift.

Steps:

    - run this notebook to upload to Redshift;
    
    - use the new panel to make calculations, new tables, dashes...
***

## 0. Setup

In [8]:
#connection to redshift
from redshift_import_export.interfaces.data_interactor import DataInteractor
di = DataInteractor()
import requests


#standard packages
import warnings
warnings.filterwarnings('ignore')
import os
import collections
import pandas as pd
import numpy as np



#time packages
import datetime as dt

## 1. Import

In [9]:
df = pd.read_csv(os.path.abspath(os.path.join(os.getcwd(), os.pardir)) + '/data/external/care_cost_por_condição_historico - Sheet1.csv')
df.head(10)

,grupo_etario,cid,total_cost,total_vol
0,0-2,A080,753,8
1,0-2,A09,45868,129
2,0-2,A38,416,8
3,0-2,B082,3552,10
4,0-2,B083,1076,13
5,0-2,B084,9784,22
6,0-2,B342,0,28
7,0-2,B35,0,9
8,0-2,B37,871,19
9,0-2,B373,1457,19


In [10]:
df.rename(columns = {'cid': 'disease_code_value', 'total_vol': 'volumetria'}, inplace = True)
df2= df[['grupo_etario', 'disease_code_value','volumetria', 'total_cost']]
df2.head(10)

,grupo_etario,disease_code_value,volumetria,total_cost
0,0-2,A080,8,753
1,0-2,A09,129,45868
2,0-2,A38,8,416
3,0-2,B082,10,3552
4,0-2,B083,13,1076
5,0-2,B084,22,9784
6,0-2,B342,28,0
7,0-2,B35,9,0
8,0-2,B37,19,871
9,0-2,B373,19,1457


## 3. Export

In [11]:
di.redshift.insert_table(df2, table_name='care_cost_for_dalys_calculation', if_exists='replace')

Table care_cost_for_dalys_calculation inserted successfully, with 2663 rows.
